In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import re
import duckdb
import torch
from transformers import BertTokenizer, BertModel, DistilBertTokenizer, DistilBertModel
from tqdm import tqdm 
import unicodedata

In [3]:
def preprocess_imdb_data(data_path, directors_path, writers_path):
    """
    General preprocessing pipeline for IMDB data with batch processing for BERT embeddings.
    
    Arguments:
    - data_path: Path to the train/test/validation data CSV file.
    - directors_path: Path to the directing.json file.
    - writers_path: Path to the writing.json file.
    
    Returns:
    - Cleaned Pandas DataFrame ready for model training or prediction.
    """
    
    # Step 1: Load main dataset
    df = pd.read_csv(data_path)

    # Step 2: Load JSON files (Directors & Writers)
    df_directors = pd.read_json(directors_path)
    df_writers = pd.read_json(writers_path)

    # Step 3: Rename columns for consistency
    df_directors.rename(columns={"movie": "tconst", "director": "director_id"}, inplace=True)
    df_writers.rename(columns={"movie": "tconst", "writer": "writer_id"}, inplace=True)

    # Step 4: Convert nested JSON fields into strings (fixes 'unhashable type' error)
    df_directors["director_id"] = df_directors["director_id"].astype(str)
    df_writers["writer_id"] = df_writers["writer_id"].astype(str)

    # Step 5: Merge main dataset with Directors & Writers using DuckDB
    import duckdb
    con = duckdb.connect()
    con.register("movies", df)
    con.register("directors", df_directors)
    con.register("writers", df_writers)

    query = """
    SELECT 
        movies.*, 
        directors.director_id, 
        writers.writer_id
    FROM movies
    LEFT JOIN directors ON movies.tconst = directors.tconst
    LEFT JOIN writers ON movies.tconst = writers.tconst
    """

    df = con.execute(query).fetchdf()
    con.close()

        # Step 6: Create column year from startYear and endYear
    df['startYear'] = df['startYear'].replace('\\N', np.nan).astype(float)
    df['endYear'] = df['endYear'].replace('\\N', np.nan).astype(float)
    df['Year'] = df['startYear'].fillna(df['endYear'])

    # Step 7: Clean title names
    def normalize_text(text):
        if pd.isna(text):  # Handle missing values
            return ""
        text = str(text)
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')  # Remove accents
        text = re.sub(r'[^\w\s]', '', text)  # Remove any remaining special characters
        return text.strip()

    def clean_titles(row):
        primary = row['primaryTitle'] if pd.notna(row['primaryTitle']) else ''
        original = row['originalTitle'] if pd.notna(row['originalTitle']) else ''

        # If primaryTitle is empty, replace it with originalTitle
        if not primary:
            primary = original

        # Normalize primaryTitle
        cleaned_title = normalize_text(primary)

        # If both titles are missing after normalization, return "Unknown Title"
        return cleaned_title if cleaned_title else "Unknown Title"

    df['primaryTitle'] = df.apply(clean_titles, axis=1)
    df.rename(columns={'primaryTitle': 'movieTitle'}, inplace=True)

    # Step 8: Drop unnecessary columns
    columns_to_drop = ["startYear", "endYear", "originalTitle", "Unnamed: 0"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    # Step 9: Handle missing values
    numeric_columns = ["runtimeMinutes", "numVotes"]
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")  # Ensure numeric format
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())  # Fill missing with median

    df["director_id"] = df["director_id"].fillna("unknown")
    df["writer_id"] = df["writer_id"].fillna("unknown")

    # Step 10: Ensure correct data types
    df["Year"] = df["Year"].astype(int)
    df["runtimeMinutes"] = df["runtimeMinutes"].astype(int)
    df["numVotes"] = df["numVotes"].astype(int)
    df["movieTitle"] = df["movieTitle"].astype(str)

    # Step 11: Ensure each `tconst` is unique
    df = df.groupby("tconst").first().reset_index()

    # Step 12: Count words in each title
    df["word_count"] = df["movieTitle"].apply(lambda x: len(x.split()))

    return df

In [4]:
# Define file paths
# Define the base directory
base_data_dir = os.path.join(os.getcwd(), "imdb")

# Generate the list of train file paths
train_files = [os.path.join(base_data_dir, f) for f in os.listdir(base_data_dir) if f.startswith("train-") and f.endswith(".csv")]

# Define paths for directors and writers files
directors_path = os.path.join(base_data_dir, "directing.json")
writers_path = os.path.join(base_data_dir, "writing.json")

# Load JSON files (Directors & Writers)
df_directors = pd.read_json(directors_path)
df_writers = pd.read_json(writers_path)

# Preprocess and merge all training data
df_train = pd.concat([preprocess_imdb_data(file, directors_path, writers_path) for file in train_files], ignore_index=True)

# Preprocess validation and test data
df_val = preprocess_imdb_data(os.path.join(base_data_dir, "validation_hidden.csv"), directors_path, writers_path)
df_test = preprocess_imdb_data(os.path.join(base_data_dir, "test_hidden.csv"), directors_path, writers_path)

# Save cleaned datasets
df_train.to_csv("cleaned/final_training_dataTitleFeatures.csv", index=False)
df_val.to_csv("cleaned/final_validation_dataTitleFeatures.csv", index=False)
df_test.to_csv("cleaned/final_test_dataTitle.csv", index=False)

print("\n✅ All datasets have been preprocessed and saved!")



✅ All datasets have been preprocessed and saved!


In [5]:
import pandas as pd
import numpy as np
import torch
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Split df_train into train and validation (80/20 split)
df_train_small, df_val_small = train_test_split(df_train, test_size=0.2, random_state=42, stratify=df_train["label"])

# Define Features (NO BERT EMBEDDINGS, ONLY STRUCTURED DATA)
features = ["Year", "runtimeMinutes", "numVotes", "director_id", "writer_id", "word_count"]
X_train_small = df_train_small[features]
X_val_small = df_val_small[features]
y_train_small = df_train_small["label"]
y_val_small = df_val_small["label"]

# ✅ Check if all features exist
print("Existing features in X_train_small:", X_train_small.columns.tolist())

# Preprocessing Pipeline
numeric_features = ["Year", "runtimeMinutes", "numVotes"]
categorical_features = ["director_id", "writer_id"]

# ✅ Fix: Handle unknown categories in `OrdinalEncoder`
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), categorical_features)  # ✅ Fix applied
    ]
)

# Train RandomForest Model
model = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))
])

print("🔹 Training model on 80% of train data...")
model.fit(X_train_small, y_train_small)

# Generate Predictions on Validation Set
y_val_pred = model.predict(X_val_small)

# Compute Evaluation Metrics
accuracy = accuracy_score(y_val_small, y_val_pred)
precision = precision_score(y_val_small, y_val_pred, average="binary")
recall = recall_score(y_val_small, y_val_pred, average="binary")
f1 = f1_score(y_val_small, y_val_pred, average="binary")

# Print Results
print("\n📊 Model Evaluation on Train Data Split (80/20)")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1 Score: {f1:.4f}")


Existing features in X_train_small: ['Year', 'runtimeMinutes', 'numVotes', 'director_id', 'writer_id', 'word_count']
🔹 Training model on 80% of train data...

📊 Model Evaluation on Train Data Split (80/20)
✅ Accuracy: 0.5496
✅ Precision: 0.5287
✅ Recall: 0.9336
✅ F1 Score: 0.6751


In [4]:
# Define file paths
# Define the base directory
base_data_dir = os.path.join(os.getcwd(), "imdb")

# Generate the list of train file paths
train_files = [os.path.join(base_data_dir, f) for f in os.listdir(base_data_dir) if f.startswith("train-") and f.endswith(".csv")]

# Define paths for directors and writers files
directors_path = os.path.join(base_data_dir, "directing.json")
writers_path = os.path.join(base_data_dir, "writing.json")

# Load JSON files (Directors & Writers)
df_directors = pd.read_json(directors_path)
df_writers = pd.read_json(writers_path)

# Preprocess and merge all training data
df_train = pd.concat([preprocess_imdb_data(file, directors_path, writers_path) for file in train_files], ignore_index=True)

# Preprocess validation and test data
df_val = preprocess_imdb_data(os.path.join(base_data_dir, "validation_hidden.csv"), directors_path, writers_path)
df_test = preprocess_imdb_data(os.path.join(base_data_dir, "test_hidden.csv"), directors_path, writers_path)

# Save cleaned datasets
df_train.to_csv("cleaned/final_training_dataTitleFeatures.csv", index=False)
df_val.to_csv("cleaned/final_validation_dataTitleFeatures.csv", index=False)
df_test.to_csv("cleaned/final_test_dataTitle.csv", index=False)

print("\n✅ All datasets have been preprocessed and saved!")


Processing DistilBERT Embeddings: 100%|████████| 34/34 [00:09<00:00,  3.66it/s]



✅ All datasets have been preprocessed and saved!


In [14]:
import pandas as pd
import numpy as np
import torch
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

# # Define Features & Target
# bert_columns = [f"bert_{i}" for i in range(df_train["distilbert_embedding"][0].shape[0])]

# # Expand BERT embeddings
# bert_train = pd.DataFrame(df_train["distilbert_embedding"].tolist(), columns=bert_columns)
# bert_val = pd.DataFrame(df_val["distilbert_embedding"].tolist(), columns=bert_columns)
# bert_test = pd.DataFrame(df_test["distilbert_embedding"].tolist(), columns=bert_columns)

# # Reset index
# bert_train.index = df_train.index
# bert_val.index = df_val.index
# bert_test.index = df_test.index

# # Merge expanded embeddings
# df_train = pd.concat([df_train, bert_train], axis=1)
# df_val = pd.concat([df_val, bert_val], axis=1)
# df_test = pd.concat([df_test, bert_test], axis=1)

# Define features
features = ["Year", "runtimeMinutes", "numVotes", "director_id", "writer_id", "word_count"]
X_train = df_train[features]
y_train = df_train["label"]
X_val = df_val[features]
X_test = df_test[features]

# Preprocessing Pipeline
numeric_features = ["Year", "runtimeMinutes", "numVotes"]
categorical_features = ["director_id", "writer_id"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OrdinalEncoder(), categorical_features)
    ]
)

# Train RandomForest Model
model = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))
])

print("🔹 Training model on full training data...")
model.fit(X_train, y_train)

# Generate Predictions
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Save predictions in required format (no headers, single column)
pd.DataFrame(y_val_pred).to_csv("submissions/validation_predictions_TitleFeatures.csv", index=False, header=False)
pd.DataFrame(y_test_pred).to_csv("submissions/test_predictions_TitleFeatures.csv", index=False, header=False)

print("✅ Predictions saved for submission!")

🔹 Training model on full training data...


ValueError: Found unknown categories ['nm0760196', 'nm0770234', 'nm2493041', 'nm0056030', 'nm0827854', 'nm0475852', 'nm0186091', 'nm0001692', 'nm0797869', 'nm0885108', 'nm0000578', 'nm0272568', 'nm1957032', 'nm0242757', 'nm1260607', 'nm0464180', 'nm2797828', 'nm1223294', 'nm0605266', 'nm0587518', 'nm1624755', 'nm0473329', 'nm1061993', 'nm1000159', 'nm0569891', 'nm2669989', 'nm3306459', 'nm5107883', 'nm6016411', 'nm0431172', 'nm0079387', 'nm1811747', 'nm0718469', 'nm0648740', 'nm0110564', 'nm0533666', 'nm0998110', 'nm0639858', 'nm0423646', 'nm1048540', 'nm8381668', 'nm0522028', 'nm3201804', 'nm4033941', 'nm4980768', 'nm0941262', 'nm1242277', 'nm3155648', 'nm0005197', 'nm0294457', 'nm0801885', 'nm1868917', 'nm1473037', 'nm1936947', 'nm1361273', 'nm1804614', 'nm0762517', 'nm1970598', 'nm0103971', 'nm0508998', 'nm2655364', 'nm0526019', 'nm0446819', 'nm0001774', 'nm0672769', 'nm2258148', 'nm0899121', 'nm0075263', 'nm0394233', 'nm0411259', 'nm3052903', 'nm6174515', 'nm4909815', 'nm3184677', 

In [13]:
import pandas as pd
import numpy as np
import torch
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Split df_train into train and validation (80/20 split)
df_train_small, df_val_small = train_test_split(df_train, test_size=0.2, random_state=42, stratify=df_train["label"])

# # Define Features
# bert_columns = [f"bert_{i}" for i in range(df_train["distilbert_embedding"][0].shape[0])]

# # Expand BERT embeddings
# bert_train_small = pd.DataFrame(df_train_small["distilbert_embedding"].tolist(), columns=bert_columns)
# bert_val_small = pd.DataFrame(df_val_small["distilbert_embedding"].tolist(), columns=bert_columns)

# # Reset index
# bert_train_small.index = df_train_small.index
# bert_val_small.index = df_val_small.index

# # Merge expanded embeddings
# df_train_small = pd.concat([df_train_small, bert_train_small], axis=1)
# df_val_small = pd.concat([df_val_small, bert_val_small], axis=1)

# Define Features & Target
features = ["Year", "runtimeMinutes", "numVotes", "director_id", "writer_id", "word_count"]
X_train_small = df_train_small[features]
y_train_small = df_train_small["label"]
X_val_small = df_val_small[features]
y_val_small = df_val_small["label"]

# ✅ Check if all features exist
print("Existing features in X_train_small:", X_train_small.columns.tolist())

# Preprocessing Pipeline
numeric_features = ["Year", "runtimeMinutes", "numVotes"]
categorical_features = ["director_id", "writer_id"]

# ✅ Fix: Handle unknown categories in `OrdinalEncoder`
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), categorical_features)  # ✅ Fix applied
    ]
)

# Train RandomForest Model
model = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))
])

print("🔹 Training model on 80% of train data...")
model.fit(X_train_small, y_train_small)

# Generate Predictions on Validation Set
y_val_pred = model.predict(X_val_small)

# Compute Evaluation Metrics
accuracy = accuracy_score(y_val_small, y_val_pred)
precision = precision_score(y_val_small, y_val_pred, average="binary")
recall = recall_score(y_val_small, y_val_pred, average="binary")
f1 = f1_score(y_val_small, y_val_pred, average="binary")

# Print Results
print("\n📊 Model Evaluation on Train Data Split (80/20)")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1 Score: {f1:.4f}")


Existing features in X_train_small: ['Year', 'runtimeMinutes', 'numVotes', 'director_id', 'writer_id']
🔹 Training model on 80% of train data...

📊 Model Evaluation on Train Data Split (80/20)
✅ Accuracy: 0.5496
✅ Precision: 0.5287
✅ Recall: 0.9336
✅ F1 Score: 0.6751
